<a href="https://colab.research.google.com/github/MeeGyo/dataviz2025/blob/main/student_health_warehouse_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Lab: สร้าง Health Data Warehouse (ฉบับนักศึกษา)

**ชื่อ**: นางสาวภาวิณี หน่อสีดา  **รหัสนักศึกษา**: 663020288-2

## 🎯 วัตถุประสงค์:
ฝึกปฏิบัติการสร้าง Data Warehouse จากฐานข้อมูลระบบเฝ้าระวังโรค NCDs

## 📚 สิ่งที่จะได้เรียนรู้:
1. การเชื่อมต่อและดึงข้อมูลจาก SQLite Database
2. การทำ Data Transformation (JOIN, Groupby, Merge)
3. การสร้าง Data Warehouse 5 ตาราง
4. การอัปโหลดข้อมูลไปยัง Google Sheets

## ⏱️ เวลาที่ใช้: 120 นาที

---

## 📦 Part 1: เตรียมสภาพแวดล้อม (10 นาที)

### Exercise 1.1: ติดตั้ง Libraries

**โจทย์**: ติดตั้ง packages ที่จำเป็นสำหรับ Lab นี้

**Hint**: ต้องติดตั้ง `gspread`, `google-auth`, `google-auth-oauthlib`, `google-auth-httplib2`, `openpyxl`

In [1]:
# TODO: ติดตั้ง packages ที่จำเป็น
# Hint: ใช้ !pip install ... -q

# เขียนโค้ดที่นี่
!pip install gspread oauth2client google-auth-oauthlib google-auth-httplib2 openpyxl -q
print("✅ ติดตั้ง libraries สำเร็จ")

✅ ติดตั้ง libraries สำเร็จ


### Exercise 1.2: Mount Google Drive

**โจทย์**: Mount Google Drive และตรวจสอบว่ามีไฟล์ที่ต้องการหรือไม่

**ไฟล์ที่ต้องการ**:
- `ncds_sample.db`
- `districts_with_zone_20250930_084409.xlsx`

**Path**: `/content/drive/MyDrive/Dashboard-Teaching-2025/`

In [20]:
# TODO: Mount Google Drive
# Hint: ใช้ from google.colab import drive
#       drive.mount('/content/drive')

# เขียนโค้ดที่นี่
from google.colab import drive
drive.mount('/content/drive')

# TODO: กำหนด path ของไฟล์
import os

SHARED_FOLDER = '/content/drive/MyDrive/Python&DataViz2025'
DB_PATH = os.path.join(SHARED_FOLDER, 'ncds_sample.db')
DISTRICTS_PATH = os.path.join(SHARED_FOLDER, 'districts_with_zone_20250930_084409.xlsx')

# TODO: ตรวจสอบว่าไฟล์มีอยู่จริง
# Hint: ใช้ os.path.exists()

# เขียนโค้ดที่นี่
print(f"✅ พบไฟล์ database" if os.path.exists(DB_PATH) else "❌ ไม่พบไล์ database")
print(f"✅ พบไฟล์ข้อมูลพื้นที่" if os.path.exists(DISTRICTS_PATH) else "❌ ไม่พบไฟล์ข้อมูลพื้นที่" )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ พบไฟล์ database
✅ พบไฟล์ข้อมูลพื้นที่


### Exercise 1.3: Import Libraries

**โจทย์**: Import libraries ที่จำเป็นทั้งหมด

In [7]:
# TODO: Import libraries ที่จำเป็น
# Hint: sqlite3, pandas, numpy, datetime, gspread, warnings

# เขียนโค้ดที่นี่
from datetime import datetime
import numpy as np
import pandas as pd
import sqlite3
import warnings
import gspread
from google.colab import auth
from google.auth import default

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("✅ Import libraries สำเร็จ")

✅ Import libraries สำเร็จ


---

## 🔌 Part 2: เชื่อมต่อและดึงข้อมูล (15 นาที)

### Exercise 2.1: เชื่อมต่อ SQLite Database

**โจทย์**: เชื่อมต่อกับ `ncds_sample.db` และแสดงรายชื่อตารางทั้งหมดพร้อมจำนวนแถว

In [15]:
# TODO: เชื่อมต่อ SQLite Database
# Hint: ใช้ sqlite3.connect(DB_PATH)

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

print("✅ เชื่อมต่อ SQLite Database สำเร็จ")
print('\nชื่อตารางทั้งหมด')
# TODO: ดูรายชื่อตารางทั้งหมดพร้อมจำนวนแถว
# Hint: SELECT name FROM sqlite_master WHERE type='table'
#       จากนั้น SELECT COUNT(*) FROM {table_name}

# เขียนโค้ดที่นี่
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
for table in tables:
    table_name = table[0]
    query = f"SELECT COUNT(*) FROM {table_name}"
    count = cursor.execute(query).fetchone()[0]
    print(f"✅ {table_name}: {count} แถว")

✅ เชื่อมต่อ SQLite Database สำเร็จ

ชื่อตารางทั้งหมด
✅ campaign: 20 แถว
✅ ncds_profile: 192 แถว
✅ ncds_record: 192 แถว
✅ ncds_habit: 192 แถว
✅ memberdata: 1920 แถว


### Exercise 2.2: ดึงข้อมูลจากตาราง

**โจทย์**: ดึงข้อมูลจาก 5 ตารางหลัก:
1. `ncds_profile`
2. `ncds_record`
3. `ncds_habit`
4. `campaign`
5. `memberdata`

**Hint**: ใช้ `pd.read_sql_query("SELECT * FROM table_name", conn)`

In [23]:
# TODO: ดึงข้อมูลจากแต่ละตาราง
print("📥 กำลังดึงข้อมูลจาก Database...")

# 1. ncds_profile
ncds_profile = pd.read_sql_query("SELECT * FROM ncds_profile", conn)
print(f"✅ ncds_profile: {len(ncds_profile)} แถว")

# 2. ncds_record
ncds_record = pd.read_sql_query("SELECT * FROM ncds_record", conn)
print(f"✅ ncds_record: {len(ncds_record)} แถว")

# 3. ncds_habit
ncds_habit = pd.read_sql_query("SELECT * FROM ncds_habit", conn)
print(f"✅ ncds_habit: {len(ncds_habit)} แถว")

# 4. campaign
campaign = pd.read_sql_query("SELECT * FROM campaign", conn)
print(f"✅ campaign: {len(campaign)} แถว")

# 5. memberdata
memberdata = pd.read_sql_query("SELECT * FROM memberdata", conn)
print(f"✅ memberdata: {len(memberdata)} แถว")

📥 กำลังดึงข้อมูลจาก Database...
✅ ncds_profile: 192 แถว
✅ ncds_record: 192 แถว
✅ ncds_habit: 192 แถว
✅ campaign: 20 แถว
✅ memberdata: 1920 แถว


### Exercise 2.3: โหลดข้อมูลพื้นที่

**โจทย์**: โหลดข้อมูลพื้นที่จากไฟล์ Excel และแปลง `district_code` เป็น string

In [24]:
# TODO: โหลดข้อมูลพื้นที่
# Hint: ใช้ pd.read_excel()

districts_df = pd.read_excel(DISTRICTS_PATH)
districts_df['district_code'] = districts_df['district_code'].astype(str)

print(f"✅ โหลดข้อมูลพื้นที่: {len(districts_df)} แถว")
districts_df.head()

✅ โหลดข้อมูลพื้นที่: 7356 แถว


,district_code,district_name,amphur_code,amphur_name,province_code,province_name,lat,lng,zone
0,100101,แขวง พระบรมมหาราชวัง,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.751,100.492,4.0
1,100102,แขวง วังบูรพาภิรมย์,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.744,100.499,4.0
2,100103,แขวง วัดราชบพิธ,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.750,100.499,4.0
3,100104,แขวง สำราญราษฎร์,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.751,100.503,4.0
4,100105,แขวง ศาลเจ้าพ่อเสือ,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.754,100.497,4.0


In [34]:
print(latest_habits.columns)

Index(['id', '2q_1', '2q_2', 'alcohol', 'concentration_problem', 'dessert',
       'exercise', 'fat_food', 'fish_sauce', 'gourmet_powder', 'instant_food',
       'irritable', 'movement', 'processed_food', 'sleep', 'sleep_problem',
       'smoking', 'sweet_drink', 'sweet_food', 'vegetable', 'dateadd',
       'memberid', 'bored', 'detached', 'campaignId'],
      dtype='object')


---

## 🏗️ Part 3: สร้าง Data Warehouse (40 นาที)

### Exercise 3.1: Health Profile Warehouse

**โจทย์**: สร้างตาราง Health Profile Warehouse ตามขั้นตอนต่อไปนี้:

**ขั้นตอน**:
1. ดึงข้อมูล**ล่าสุด**ของแต่ละสมาชิกจาก `ncds_profile` และ `ncds_record`
2. นับจำนวนครั้งการตรวจของแต่ละสมาชิก
3. JOIN กับ `campaign` เพื่อได้ `district_code`
4. JOIN กับ `districts_df` เพื่อได้ข้อมูลพื้นที่
5. สร้าง DataFrame ตามคอลัมน์ที่กำหนด

**คอลัมน์ที่ต้องมี**:
- member_id
- จำนวนครั้งการตรวจ
- เพศ (แปลง 0→หญิง, 1→ชาย)
- อายุ
- ตำบล_id, ตำบล
- อำเภอ_id, อำเภอ
- จังหวัด_id, จังหวัด
- เขต
- กลุ่มอายุ
- ประวัติการเจ็บป่วย (code และ label)
- ประวัติแพ้ยา (code และ label)
- ประวัติแพ้อาหาร (code และ label)
- ญาติสายตรงNCDs (code และ label)

In [25]:
# Step 1: ดึงข้อมูลล่าสุดของแต่ละสมาชิก
# Hint: ใช้ .sort_values('dateadd').groupby('memberid').tail(1)

latest_profiles = ncds_profile.sort_values('dateadd').groupby('memberid').tail(1)
latest_records = ncds_record.sort_values('dateadd').groupby('memberid').tail(1)

print(f"✓ Latest profiles: {len(latest_profiles)} แถว")
print(f"✓ Latest records: {len(latest_records)} แถว")

✓ Latest profiles: 100 แถว
✓ Latest records: 100 แถว


In [26]:
# Step 2: นับจำนวนครั้งการตรวจ
# Hint: ใช้ .groupby('memberid').size().reset_index(name='จำนวนครั้งการตรวจ')

screening_count = ncds_record.groupby('memberid').size().reset_index(name='จำนวนครั้งการตรวจ')

print(f"✓ Screening count: {len(screening_count)} สมาชิก")
screening_count.head()

✓ Screening count: 100 สมาชิก


,memberid,จำนวนครั้งการตรวจ
0,10000000,3
1,10000001,3
2,10000002,3
3,10000003,2
4,10000004,2


In [27]:
# Step 3: JOIN กับ campaign
# Hint: ใช้ pd.merge() โดย left_on='campaignId', right_on='id'

campaign['district_code'] = campaign['district_code'].astype(str)

profile_campaign = pd.merge(latest_profiles, campaign[['id', 'district_code']],
                            left_on='campaignId', right_on='id', how='left')

print(f"✓ Profile + Campaign: {len(profile_campaign)} แถว")

✓ Profile + Campaign: 100 แถว


In [28]:
# Step 4: JOIN กับ districts
# Hint: ใช้ pd.merge() โดย on='district_code'

profile_location = pd.merge(profile_campaign, districts_df, on='district_code', how='left')

print(f"✓ Profile + Location: {len(profile_location)} แถว")

✓ Profile + Location: 100 แถว


In [29]:
# Step 5: JOIN กับ record และ screening_count
# Hint: JOIN 2 ครั้ง - ครั้งแรกกับ latest_records, ครั้งที่สองกับ screening_count

profile_record = pd.merge(profile_location, latest_records, on='memberid', how='left')

final_data = pd.merge(profile_record, screening_count, on='memberid', how='left')

# แปลง NaN ในจำนวนครั้งการตรวจเป็น 0
final_data['จำนวนครั้งการตรวจ'] = final_data['จำนวนครั้งการตรวจ'].fillna(0).astype(int)

print(f"✓ Final data: {len(final_data)} แถว")
final_data.head()

✓ Final data: 100 แถว


,id_x,dateadd_x,detail,memberid,age,gender,campaignId,risk_bmi,risk_habit,risk_waistline,risk_bloodpressure,risk_fbs,risk_movement,risk_sleep,risk_2q,risk_stress,risk_vegetable,risk_sugar,risk_fat,risk_sodium,id_y,district_code,district_name,amphur_code,amphur_name,province_code,province_name,lat,lng,zone,id,dateadd_y,drug_allergy,drug_allergy_note,food_allergy,food_allergy_note,record_coronary_artery_disease,record_emphysema,record_ncds,record_obesity,relatives_chronic_kidney_disease,relatives_emphysema,relatives_ncds,campaignid,record_cancer,record_chronic_kidney_disease,record_cirrhosis,record_dementia,record_diabetes,record_hyperlipidemia,record_hypertension,record_stroke,relatives_cancer,relatives_cirrhosis,relatives_coronary_artery_disease,relatives_dementia,relatives_diabetes,relatives_hyperlipidemia,relatives_hypertension,relatives_stroke,relatives_obesity,จำนวนครั้งการตรวจ
0,77,2025-06-02 00:00:00,0b654fa141f37aee35abf9d3555527b94bf7b9b2e167dc...,10000036,50,1,100008,low,medium,medium,high,,high,high,,medium,high,high,,medium,100008,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,18.796,98.985,2.0,77,2025-06-02 00:00:00,1,ยาปฏิชีวนะ,-1,,0,0,-1,0,0,0,-1,100008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1
1,114,2025-06-10 00:00:00,da3b431e84ad23aecc05a3a6ca86c0aa3ee563a21e6b23...,10000058,44,0,100006,high,medium,low,high,low,low,low,,low,low,medium,medium,high,100006,120102,ต. ตลาดขวัญ,1201.0,อ. เมืองนนทบุรี,12.0,จ. นนทบุรี,13.850,100.509,6.0,114,2025-06-10 00:00:00,-1,,-1,,0,0,-1,0,0,0,-1,100006,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2
2,54,2025-06-12 00:00:00,ffd144bbca0915dedc29ff24a4990a2a4a08ed615f3b98...,10000025,73,0,100015,high,medium,high,low,medium,medium,low,high,medium,medium,medium,low,low,100015,400101,ต. ในเมือง,4001.0,อ. เมืองขอนแก่น,40.0,จ. ขอนแก่น,16.438,102.837,1.0,54,2025-06-12 00:00:00,1,เพนนิซิลลิน,-1,,0,0,-1,1,1,0,-1,100015,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1
3,82,2025-06-12 00:00:00,a33f5d57dd594b547ab361798301edb6a7075de5f9496a...,10000040,48,0,100008,low,low,,low,high,high,high,low,low,high,medium,high,high,100008,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,18.796,98.985,2.0,82,2025-06-12 00:00:00,-1,,-1,,0,0,-1,1,0,0,1,100008,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,142,2025-06-13 00:00:00,2ccc87026f02cff661c222f46039fb18cbc46d0900fce4...,10000076,62,0,100001,medium,low,medium,low,high,,low,medium,high,,low,high,,100001,100101,แขวง พระบรมมหาราชวัง,1001.0,เขต พระนคร,10.0,กรุงเทพมหานคร,13.751,100.492,4.0,142,2025-06-13 00:00:00,-1,,-1,,0,0,-1,0,0,0,-1,100001,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [30]:
# Step 6: สร้าง Helper Functions
# TODO: เขียนฟังก์ชันแปลงค่าต่างๆ

def get_gender_label(gender):
    """แปลงเพศ: 0→หญิง, 1→ชาย, อื่นๆ→999"""
    # เขียนโค้ดที่นี่
    if gender == 0:
        return 'หญิง'
    elif gender == 1:
        return 'ชาย'
    else:
        return 'ไม่ทราบ'

def get_age_group(age):
    """
    แปลงกลุ่มอายุ:
    - < 20: 'อายุต่ำกว่า 20 ปี'
    - 20-34: 'อายุระหว่าง 20-34 ปี'
    - 35-59: 'อายุระหว่าง 35-59 ปี'
    - >= 60: 'อายุ 60 ปีขึ้นไป'
    """
    # เขียนโค้ดที่นี่
    if age < 20:
        return 'อายุต่ำกว่า 20 ปี'
    elif 20 <= age <= 34:
        return 'อายุระหว่าง 20-34 ปี'
    elif 35 <= age <= 59:
        return 'อายุระหว่าง 35-59 ปี'
    elif age >= 60:
        return 'อายุ 60 ปีขึ้นไป'
    else:
        return 'ไม่ทราบ'


def convert_illness_history(value):
    """
    แปลงประวัติการเจ็บป่วย:
    - -1: 'ไม่มีประวัติ'
    - 1: 'มีประวัติการเจ็บป่วยรวม'
    - อื่นๆ: 'ไม่ทราบ'
    """
    # เขียนโค้ดที่นี่
    if value == -1:
        return 'ไม่มีประวัติ'
    elif value == 1:
        return 'มีประวัติการเจ็บป่วยรวม'
    else:
        return 'ไม่ทราบ'

# TODO: เขียนฟังก์ชันเพิ่มเติมสำหรับ drug_allergy, food_allergy, relatives_ncds
# (รูปแบบเหมือน convert_illness_history)

def convert_allergy(value):
    """
    แปลงประวัติแพ้ (ยา/อาหาร):
    - -1: 'ไม่มีประวัติ'
    - 1: 'มีประวัติแพ้'
    - อื่นๆ: 'ไม่ทราบ'
    """
    if value == -1:
        return 'ไม่มีประวัติ'
    elif value == 1:
        return 'มีประวัติแพ้'
    else:
        return 'ไม่ทราบ'

def convert_relatives_ncds(value):
    """
    แปลงประวัติญาติสายตรงNCDs:
    - -1: 'ไม่มีประวัติ'
    - 1: 'มีประวัติ'
    - อื่นๆ: 'ไม่ทราบ'
    """
    if value == -1:
        return 'ไม่มีประวัติ'
    elif value == 1:
        return 'มีประวัติ'
    else:
        return 'ไม่ทราบ'

In [31]:
# Step 7: สร้าง Health Profile Warehouse
# TODO: สร้าง DataFrame ตามคอลัมน์ที่กำหนด

health_profile_wh = pd.DataFrame({
    'member_id': final_data['memberid'],
    'จำนวนครั้งการตรวจ': final_data['จำนวนครั้งการตรวจ'],
    'เพศ': final_data['gender'].apply(get_gender_label),
    'อายุ': final_data['age'],
    'ตำบล_id': final_data['district_code'],
    'ตำบล': final_data['district_name'],
    'อำเภอ_id': final_data['amphur_code'],
    'อำเภอ': final_data['amphur_name'],
    'จังหวัด_id': final_data['province_code'],
    'จังหวัด': final_data['province_name'],
    'เขต': final_data['zone'],
    'กลุ่มอายุ': final_data['age'].apply(get_age_group),
    'ประวัติการเจ็บป่วย': final_data['record_ncds'].apply(convert_illness_history),
    'ประวัติแพ้ยา': final_data['drug_allergy'].apply(convert_allergy),
    'ประวัติแพ้อาหาร': final_data['food_allergy'].apply(convert_allergy),
    'ญาติสายตรงNCDs': final_data['relatives_ncds'].apply(convert_relatives_ncds)
})


print(f"✅ Health Profile Warehouse: {len(health_profile_wh)} แถว")
display(health_profile_wh.head())

✅ Health Profile Warehouse: 100 แถว


,member_id,จำนวนครั้งการตรวจ,เพศ,อายุ,ตำบล_id,ตำบล,อำเภอ_id,อำเภอ,จังหวัด_id,จังหวัด,เขต,กลุ่มอายุ,ประวัติการเจ็บป่วย,ประวัติแพ้ยา,ประวัติแพ้อาหาร,ญาติสายตรงNCDs
0,10000036,1,ชาย,50,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,2.0,อายุระหว่าง 35-59 ปี,ไม่มีประวัติ,มีประวัติแพ้,ไม่มีประวัติ,ไม่มีประวัติ
1,10000058,2,หญิง,44,120102,ต. ตลาดขวัญ,1201.0,อ. เมืองนนทบุรี,12.0,จ. นนทบุรี,6.0,อายุระหว่าง 35-59 ปี,ไม่มีประวัติ,ไม่มีประวัติ,ไม่มีประวัติ,ไม่มีประวัติ
2,10000025,1,หญิง,73,400101,ต. ในเมือง,4001.0,อ. เมืองขอนแก่น,40.0,จ. ขอนแก่น,1.0,อายุ 60 ปีขึ้นไป,ไม่มีประวัติ,มีประวัติแพ้,ไม่มีประวัติ,ไม่มีประวัติ
3,10000040,1,หญิง,48,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,2.0,อายุระหว่าง 35-59 ปี,ไม่มีประวัติ,ไม่มีประวัติ,ไม่มีประวัติ,มีประวัติ
4,10000076,1,หญิง,62,100101,แขวง พระบรมมหาราชวัง,1001.0,เขต พระนคร,10.0,กรุงเทพมหานคร,4.0,อายุ 60 ปีขึ้นไป,ไม่มีประวัติ,ไม่มีประวัติ,ไม่มีประวัติ,ไม่มีประวัติ


### Exercise 3.2: NCDs Disease Warehouse

**โจทย์**: สร้างตารางประวัติโรค NCDs

**ขั้นตอน**:
1. ดึงข้อมูลล่าสุดจาก `ncds_record`
2. สำหรับแต่ละสมาชิก สร้างรายการโรคของผู้ตรวจ (8 โรค)
3. สร้างรายการโรคของญาติ (9 โรค)
4. รวมเป็นตารางเดียว โดยแต่ละแถว = 1 โรค

**โรคของผู้ตรวจ**: cancer, chronic_kidney_disease, cirrhosis, dementia, diabetes, hyperlipidemia, hypertension, stroke

**โรคของญาติ**: cancer, cirrhosis, coronary_artery_disease, dementia, diabetes, hyperlipidemia, hypertension, stroke, obesity

In [32]:
# TODO: สร้าง NCDs Disease Warehouse
# Hint: ดูตัวอย่างจากไฟล์ detailed_health_warehouse_tutorial.ipynb

# ดึงข้อมูลล่าสุดจาก ncds_record
latest_records = ncds_record.sort_values('dateadd').groupby('memberid').tail(1).reset_index(drop=True)

# กำหนดรายการโรค
illness_cols = [
    'record_cancer', 'record_chronic_kidney_disease', 'record_cirrhosis',
    'record_dementia', 'record_diabetes', 'record_hyperlipidemia',
    'record_hypertension', 'record_stroke'
]
relative_illness_cols = [
    'relatives_cancer', 'relatives_cirrhosis', 'relatives_coronary_artery_disease',
    'relatives_dementia', 'relatives_diabetes', 'relatives_hyperlipidemia',
    'relatives_hypertension', 'relatives_stroke', 'relatives_obesity'
]

# สร้าง DataFrame สำหรับโรคของผู้ตรวจ
examiner_diseases = []
for index, row in latest_records.iterrows():
    member_id = row['memberid']
    for col in illness_cols:
        if row[col] == 1:
            disease_name = col.replace('record_', '').replace('_', ' ').title()
            examiner_diseases.append({
                'member_id': member_id,
                'ประเภท': 'ผู้ตรวจ',
                'ชื่อโรค': disease_name
            })

# สร้าง DataFrame สำหรับโรคของญาติ
relative_diseases = []
for index, row in latest_records.iterrows():
    member_id = row['memberid']
    for col in relative_illness_cols:
        if row[col] == 1:
            disease_name = col.replace('relatives_', '').replace('_', ' ').title()
            relative_diseases.append({
                'member_id': member_id,
                'ประเภท': 'ญาติ',
                'ชื่อโรค': disease_name
            })

# รวม DataFrame ทั้งสอง
ncds_disease_wh = pd.DataFrame(examiner_diseases + relative_diseases)

print(f"✅ NCDs Disease Warehouse: {len(ncds_disease_wh)} แถว")
display(ncds_disease_wh.head())

✅ NCDs Disease Warehouse: 496 แถว


,member_id,ประเภท,ชื่อโรค
0,10000058,ผู้ตรวจ,Cancer
1,10000058,ผู้ตรวจ,Dementia
2,10000058,ผู้ตรวจ,Hypertension
3,10000058,ผู้ตรวจ,Stroke
4,10000025,ผู้ตรวจ,Cirrhosis


### Exercise 3.3: Eating Behavior Warehouse

**โจทย์**: สร้างตารางพฤติกรรมการกิน

**ข้อมูลที่ต้องแปลง**:
- กินผัก (vegetable): 0→ไม่ทำเลย, 1→1-3 วัน, 2→4-6 วัน, 3→ทุกวัน
- กินขนมหวาน (dessert): 0→ทุกวัน, 1→4-6 วัน, 2→1-3 วัน, 3→ไม่ทำเลย (กลับด้าน!)
- ฯลฯ

In [35]:
# TODO: สร้าง Eating Behavior Warehouse
# Hint: ต้องสร้างฟังก์ชันแปลงค่าสำหรับแต่ละพฤติกรรม

# ดึงข้อมูลล่าสุดจาก ncds_habit
latest_habits = ncds_habit.sort_values('dateadd').groupby('memberid').tail(1).reset_index(drop=True)

# Helper Functions for Eating Behavior
def convert_vegetable(value):
    """กินผัก (vegetable): 0→ไม่ทำเลย, 1→1-3 วัน, 2→4-6 วัน, 3→ทุกวัน"""
    if value == 0:
        return 'ไม่ทำเลย'
    elif value == 1:
        return '1-3 วัน'
    elif value == 2:
        return '4-6 วัน'
    elif value == 3:
        return 'ทุกวัน'
    else:
        return 'ไม่ทราบ'

def convert_dessert(value):
    """กินขนมหวาน (dessert): 0→ทุกวัน, 1→4-6 วัน, 2→1-3 วัน, 3→ไม่ทำเลย (กลับด้าน!)"""
    if value == 0:
        return 'ทุกวัน'
    elif value == 1:
        return '4-6 วัน'
    elif value == 2:
        return '1-3 วัน'
    elif value == 3:
        return 'ไม่ทำเลย'
    else:
        return 'ไม่ทราบ'

def convert_sweet_drink(value):
    """ดื่มน้ำหวาน (sweet_drink): 0→ไม่ทำเลย, 1→1-3 วัน, 2→4-6 วัน, 3→ทุกวัน"""
    if value == 0:
        return 'ไม่ทำเลย'
    elif value == 1:
        return '1-3 วัน'
    elif value == 2:
        return '4-6 วัน'
    elif value == 3:
        return 'ทุกวัน'
    else:
        return 'ไม่ทราบ'

def convert_fatty_food(value):
    """กินอาหารไขมันสูง (fatty_food): 0→ไม่ทำเลย, 1→1-3 วัน, 2→4-6 วัน, 3→ทุกวัน"""
    if value == 0:
        return 'ไม่ทำเลย'
    elif value == 1:
        return '1-3 วัน'
    elif value == 2:
        return '4-6 วัน'
    elif value == 3:
        return 'ทุกวัน'
    else:
        return 'ไม่ทราบ'

def convert_salty_food(value):
    """กินอาหารเค็ม (salty_food): 0→ไม่ทำเลย, 1→1-3 วัน, 2→4-6 วัน, 3→ทุกวัน"""
    if value == 0:
        return 'ไม่ทำเลย'
    elif value == 1:
        return '1-3 วัน'
    elif value == 2:
        return '4-6 วัน'
    elif value == 3:
        return 'ทุกวัน'
    else:
        return 'ไม่ทราบ'


# Create Eating Behavior Warehouse DataFrame
eating_behavior_wh = pd.DataFrame({
    'member_id': latest_habits['memberid'],
    'กินผัก_label': latest_habits['vegetable'].apply(convert_vegetable),
    'กินผัก_score': latest_habits['vegetable'],
    'กินขนมหวาน_label': latest_habits['dessert'].apply(convert_dessert),
    'กินขนมหวาน_score': latest_habits['dessert'],
    'ดื่มน้ำหวาน_label': latest_habits['sweet_drink'].apply(convert_sweet_drink),
    'ดื่มน้ำหวาน_score': latest_habits['sweet_drink'],
    'กินอาหารไขมันสูง_label': latest_habits['fat_food'].apply(convert_fatty_food),
    'กินอาหารไขมันสูง_score': latest_habits['fat_food'],
    'กินอาหารเค็ม_label': latest_habits['fish_sauce'].apply(convert_salty_food),
    'กินอาหารเค็ม_score': latest_habits['fish_sauce']
})


print(f"✅ Eating Behavior Warehouse: {len(eating_behavior_wh)} แถว")
display(eating_behavior_wh.head())

✅ Eating Behavior Warehouse: 100 แถว


,member_id,กินผัก_label,กินผัก_score,กินขนมหวาน_label,กินขนมหวาน_score,ดื่มน้ำหวาน_label,ดื่มน้ำหวาน_score,กินอาหารไขมันสูง_label,กินอาหารไขมันสูง_score,กินอาหารเค็ม_label,กินอาหารเค็ม_score
0,10000036,ทุกวัน,3,ทุกวัน,0,ทุกวัน,3,1-3 วัน,1,4-6 วัน,2
1,10000058,ไม่ทราบ,-1,ไม่ทำเลย,3,4-6 วัน,2,ไม่ทำเลย,0,ไม่ทำเลย,0
2,10000025,1-3 วัน,1,ทุกวัน,0,ไม่ทำเลย,0,ทุกวัน,3,4-6 วัน,2
3,10000040,4-6 วัน,2,ไม่ทำเลย,3,ไม่ทราบ,-1,1-3 วัน,1,4-6 วัน,2
4,10000076,ไม่ทำเลย,0,ไม่ทราบ,-1,ไม่ทราบ,-1,4-6 วัน,2,1-3 วัน,1


### Exercise 3.4: Comprehensive Behavior Warehouse

**โจทย์**: สร้างตารางพฤติกรรมรวม

**ข้อมูลที่ต้องแปลง**:
- การเคลื่อนไหว (movement)
- การสูบบุหรี่ (smoking)
- การนอนหลับ (sleep)
- ภาวะซึมเศร้า (2q_1)
- ความเครียด (2q_2)
- การดื่มแอลกอฮอล์ (alcohol)
- การออกกำลังกาย (exercise)

In [36]:
# TODO: สร้าง Comprehensive Behavior Warehouse
# Hint: คล้ายกับ Eating Behavior แต่เป็นพฤติกรรมด้านอื่น

# Helper Functions for Comprehensive Behavior
def convert_movement(value):
    """การเคลื่อนไหว (movement): 0→ไม่ทำเลย, 1→1-3 วัน, 2→4-6 วัน, 3→ทุกวัน"""
    if value == 0:
        return 'ไม่ทำเลย'
    elif value == 1:
        return '1-3 วัน'
    elif value == 2:
        return '4-6 วัน'
    elif value == 3:
        return 'ทุกวัน'
    else:
        return 'ไม่ทราบ'

def convert_smoking(value):
    """การสูบบุหรี่ (smoking): 0→ไม่สูบ, 1→สูบเป็นครั้งคราว, 2→สูบเป็นประจำ"""
    if value == 0:
        return 'ไม่สูบ'
    elif value == 1:
        return 'สูบเป็นครั้งคราว'
    elif value == 2:
        return 'สูบเป็นประจำ'
    else:
        return 'ไม่ทราบ'

def convert_sleep(value):
    """การนอนหลับ (sleep): 0→น้อยกว่า 6 ชม., 1→6-8 ชม., 2→มากกว่า 8 ชม."""
    if value == 0:
        return 'น้อยกว่า 6 ชม.'
    elif value == 1:
        return '6-8 ชม.'
    elif value == 2:
        return 'มากกว่า 8 ชม.'
    else:
        return 'ไม่ทราบ'

def convert_2q(value):
    """ภาวะซึมเศร้า/ความเครียด (2q_1, 2q_2): 0→ไม่มี, 1→มี"""
    if value == 0:
        return 'ไม่มี'
    elif value == 1:
        return 'มี'
    else:
        return 'ไม่ทราบ'

def convert_alcohol(value):
    """การดื่มแอลกอฮอล์ (alcohol): 0→ไม่ดื่ม, 1→ดื่มเป็นครั้งคราว, 2→ดื่มเป็นประจำ"""
    if value == 0:
        return 'ไม่ดื่ม'
    elif value == 1:
        return 'ดื่มเป็นครั้งคราว'
    elif value == 2:
        return 'ดื่มเป็นประจำ'
    else:
        return 'ไม่ทราบ'

def convert_exercise(value):
    """การออกกำลังกาย (exercise): 0→ไม่ทำเลย, 1→1-3 วัน, 2→4-6 วัน, 3→ทุกวัน"""
    if value == 0:
        return 'ไม่ทำเลย'
    elif value == 1:
        return '1-3 วัน'
    elif value == 2:
        return '4-6 วัน'
    elif value == 3:
        return 'ทุกวัน'
    else:
        return 'ไม่ทราบ'

# Create Comprehensive Behavior Warehouse DataFrame
comprehensive_behavior_wh = pd.DataFrame({
    'member_id': latest_habits['memberid'],
    'การเคลื่อนไหว_label': latest_habits['movement'].apply(convert_movement),
    'การเคลื่อนไหว_score': latest_habits['movement'],
    'การสูบบุหรี่_label': latest_habits['smoking'].apply(convert_smoking),
    'การสูบบุหรี่_score': latest_habits['smoking'],
    'การนอนหลับ_label': latest_habits['sleep'].apply(convert_sleep),
    'การนอนหลับ_score': latest_habits['sleep'],
    'ภาวะซึมเศร้า_label': latest_habits['2q_1'].apply(convert_2q),
    'ภาวะซึมเศร้า_score': latest_habits['2q_1'],
    'ความเครียด_label': latest_habits['2q_2'].apply(convert_2q),
    'ความเครียด_score': latest_habits['2q_2'],
    'การดื่มแอลกอฮอล์_label': latest_habits['alcohol'].apply(convert_alcohol),
    'การดื่มแอลกอฮอล์_score': latest_habits['alcohol'],
    'การออกกำลังกาย_label': latest_habits['exercise'].apply(convert_exercise),
    'การออกกำลังกาย_score': latest_habits['exercise']
})

print(f"✅ Comprehensive Behavior Warehouse: {len(comprehensive_behavior_wh)} แถว")
display(comprehensive_behavior_wh.head())

✅ Comprehensive Behavior Warehouse: 100 แถว


,member_id,การเคลื่อนไหว_label,การเคลื่อนไหว_score,การสูบบุหรี่_label,การสูบบุหรี่_score,การนอนหลับ_label,การนอนหลับ_score,ภาวะซึมเศร้า_label,ภาวะซึมเศร้า_score,ความเครียด_label,ความเครียด_score,การดื่มแอลกอฮอล์_label,การดื่มแอลกอฮอล์_score,การออกกำลังกาย_label,การออกกำลังกาย_score
0,10000036,4-6 วัน,2,ไม่ทราบ,-1,ไม่ทราบ,-1,ไม่มี,0,ไม่มี,0,ไม่ทราบ,3,ไม่ทำเลย,0
1,10000058,4-6 วัน,2,ไม่สูบ,0,6-8 ชม.,1,มี,1,มี,1,ไม่ทราบ,3,1-3 วัน,1
2,10000025,1-3 วัน,1,ไม่ทราบ,-1,6-8 ชม.,1,มี,1,ไม่ทราบ,-1,ไม่ดื่ม,0,1-3 วัน,1
3,10000040,4-6 วัน,2,ไม่สูบ,0,6-8 ชม.,1,ไม่มี,0,ไม่มี,0,ดื่มเป็นครั้งคราว,1,4-6 วัน,2
4,10000076,1-3 วัน,1,ไม่สูบ,0,มากกว่า 8 ชม.,2,มี,1,ไม่มี,0,ดื่มเป็นประจำ,2,ไม่ทำเลย,0


In [37]:
# TODO: สร้าง Disease Risk Warehouse
# Hint: 1. ดึงค่าจาก memberdata (sys/dias, bmi, waistline, height)
#       2. สร้างฟังก์ชันคำนวณความเสี่ยงทั้ง 3 ประเภท
#       3. Loop แต่ละสมาชิกและคำนวณ
#       4. สร้าง DataFrame จากผลลัพธ์

# 1. ดึงค่าจาก memberdata
member_measurements = memberdata[memberdata['name'].isin(['sys/dias', 'bmi', 'waistline', 'height'])]

# Pivot the data to get measurements as columns for each member
member_measurements_pivot = member_measurements.pivot_table(
    index='memberid',
    columns='name',
    values='value',
    aggfunc='first' # Use first value if multiple records exist for a member/name
).reset_index()

# Convert relevant columns to numeric, coercing errors
for col in ['bmi', 'waistline', 'height']:
    member_measurements_pivot[col] = pd.to_numeric(member_measurements_pivot[col], errors='coerce')

# Split sys/dias into sys and dias and convert to numeric
if 'sys/dias' in member_measurements_pivot.columns:
    sys_dias = member_measurements_pivot['sys/dias'].astype(str).str.split('/', expand=True)
    member_measurements_pivot['sys'] = pd.to_numeric(sys_dias[0], errors='coerce')
    member_measurements_pivot['dias'] = pd.to_numeric(sys_dias[1], errors='coerce')
    member_measurements_pivot = member_measurements_pivot.drop(columns=['sys/dias'])

# Merge with latest_records to get age, gender, relatives_diabetes, smoking
risk_data = pd.merge(
    latest_records[['memberid', 'drug_allergy', 'food_allergy', 'record_ncds', 'relatives_ncds',
                    'record_cancer', 'record_chronic_kidney_disease', 'record_cirrhosis',
                    'record_dementia', 'record_diabetes', 'record_hyperlipidemia',
                    'record_hypertension', 'record_stroke', 'relatives_cancer',
                    'relatives_cirrhosis', 'relatives_coronary_artery_disease',
                    'relatives_dementia', 'relatives_diabetes', 'relatives_hyperlipidemia',
                    'relatives_hypertension', 'relatives_stroke', 'relatives_obesity', 'campaignid']],
    member_measurements_pivot,
    on='memberid',
    how='left'
)

# Merge with latest_profiles to get age, gender
risk_data = pd.merge(
    risk_data,
    latest_profiles[['memberid', 'age', 'gender']],
    on='memberid',
    how='left'
)

# Merge with latest_habits to get smoking
risk_data = pd.merge(
    risk_data,
    latest_habits[['memberid', 'smoking', 'exercise', 'movement', 'sleep', 'alcohol', '2q_1', '2q_2', 'vegetable', 'dessert', 'sweet_drink', 'fat_food', 'fish_sauce']],
    on='memberid',
    how='left'
)


print(f"✓ Risk data prepared: {len(risk_data)} แถว")
display(risk_data.head())


# 2. สร้างฟังก์ชันคำนวณความเสี่ยงทั้ง 3 ประเภท

def calculate_hypertension_risk(sys, dias):
    """คำนวณความเสี่ยงโรคความดันโลหิตสูง"""
    if pd.isna(sys) or pd.isna(dias):
        return 'ไม่ทราบ', -1
    if sys >= 140 or dias >= 90:
        return 'สูง', 3
    elif sys >= 120 or dias >= 80:
        return 'ปานกลาง', 2
    else:
        return 'ต่ำ', 1


def calculate_findrisc_risk(age, bmi, waistline, sys, relatives_diabetes):
    """คำนวณความเสี่ยงโรคเบาหวานด้วย FINDRISC"""
    if pd.isna(age) or pd.isna(bmi) or pd.isna(waistline) or pd.isna(sys) or pd.isna(relatives_diabetes):
        return 'ไม่ทราบ', -1

    score = 0
    # Age
    if age >= 45 and age <= 54:
        score += 2
    elif age >= 55 and age <= 64:
        score += 3
    elif age >= 65:
        score += 4

    # BMI
    if bmi >= 25 and bmi <= 30:
        score += 1
    elif bmi > 30:
        score += 3

    # Waistline (assuming standard thresholds, adjust if needed for specific guidelines)
    # These thresholds are general, specific guidelines for Thai population might differ.
    if waistline >= 94: # Example threshold for men
         score += 3
    elif waistline >= 80: # Example threshold for women
         score += 3


    # History of high blood pressure medication
    # Assuming 'sys' value >= 140 indicates history or current high BP
    if sys >= 140:
        score += 2

    # Relatives with diabetes
    if relatives_diabetes == 1: # Assuming 1 indicates history
        score += 5

    # Risk level based on score (example thresholds, adjust if needed)
    if score < 7:
        return 'ต่ำ', score
    elif score >= 7 and score <= 11:
        return 'ปานกลาง', score
    elif score >= 12 and score <= 14:
        return 'สูง', score
    elif score >= 15:
        return 'สูงมาก', score
    else:
        return 'ไม่ทราบ', -1


def calculate_cvd_risk(age, gender, smoking, diabetes, sys, waistline, height):
    """คำนวณความเสี่ยงโรคหัวใจและหลอดเลือด (Simplified example)"""
    if pd.isna(age) or pd.isna(gender) or pd.isna(smoking) or pd.isna(diabetes) or pd.isna(sys) or pd.isna(waistline) or pd.isna(height) or height == 0:
         return 'ไม่ทราบ', -1

    # Convert smoking status to a simplified score (example)
    smoking_score = 0
    if smoking == 1: # สูบเป็นครั้งคราว
        smoking_score = 1
    elif smoking == 2: # สูบเป็นประจำ
        smoking_score = 2

    # Convert diabetes status to a score (example)
    diabetes_score = 0
    if diabetes == 1: # มีประวัติ
        diabetes_score = 2

    # Calculate Waist-to-Height Ratio (WtHR)
    whtr = waistline / height

    # Simplified risk score (example, not a clinically validated model)
    # This is a placeholder and should be replaced with a proper CVD risk model
    risk_score = (age * 0.1) + (gender * 0.5) + (smoking_score * 1.0) + (diabetes_score * 1.5) + (sys * 0.05) + (whtr * 10)

    # Determine risk level based on simplified score (example thresholds)
    if risk_score < 10:
        return 'ต่ำ', risk_score
    elif risk_score >= 10 and risk_score < 15:
        return 'ปานกลาง', risk_score
    else:
        return 'สูง', risk_score


# 3. Loop แต่ละสมาชิกและคำนวณ
disease_risk_list = []
for index, row in risk_data.iterrows():
    member_id = row['memberid']
    age = row['age']
    gender = row['gender']
    bmi = row.get('bmi')
    waistline = row.get('waistline')
    height = row.get('height')
    sys = row.get('sys')
    dias = row.get('dias')
    relatives_diabetes = row.get('relatives_diabetes')
    smoking = row.get('smoking')
    record_diabetes = row.get('record_diabetes')


    # Calculate Hypertension Risk
    ht_level, ht_score = calculate_hypertension_risk(sys, dias)
    disease_risk_list.append({
        'member_id': member_id,
        'ภาวะเสี่ยง': 'ความดันโลหิตสูง',
        'ระดับความเสี่ยง': ht_level,
        'คะแนน': ht_score
    })

    # Calculate FINDRISC (Diabetes) Risk
    findrisc_level, findrisc_score = calculate_findrisc_risk(age, bmi, waistline, sys, relatives_diabetes)
    disease_risk_list.append({
        'member_id': member_id,
        'ภาวะเสี่ยง': 'เบาหวาน (FINDRISC)',
        'ระดับความเสี่ยง': findrisc_level,
        'คะแนน': findrisc_score
    })

    # Calculate CVD Risk (Simplified)
    # Note: The simplified CVD risk function uses 'record_diabetes' from latest_records
    cvd_level, cvd_score = calculate_cvd_risk(age, gender, smoking, record_diabetes, sys, waistline, height)
    disease_risk_list.append({
        'member_id': member_id,
        'ภาวะเสี่ยง': 'โรคหัวใจและหลอดเลือด (Simplified)',
        'ระดับความเสี่ยง': cvd_level,
        'คะแนน': cvd_score
    })


# 4. สร้าง DataFrame จากผลลัพธ์
disease_risk_wh = pd.DataFrame(disease_risk_list)

print(f"✅ Disease Risk Warehouse: {len(disease_risk_wh)} แถว")
display(disease_risk_wh.head())

✓ Risk data prepared: 100 แถว


,memberid,drug_allergy,food_allergy,record_ncds,relatives_ncds,record_cancer,record_chronic_kidney_disease,record_cirrhosis,record_dementia,record_diabetes,record_hyperlipidemia,record_hypertension,record_stroke,relatives_cancer,relatives_cirrhosis,relatives_coronary_artery_disease,relatives_dementia,relatives_diabetes,relatives_hyperlipidemia,relatives_hypertension,relatives_stroke,relatives_obesity,campaignid,bmi,height,waistline,sys,dias,age,gender,smoking,exercise,movement,sleep,alcohol,2q_1,2q_2,vegetable,dessert,sweet_drink,fat_food,fish_sauce
0,10000036,1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,100008,33.8,167,110,157,95,50,1,-1,0,2,-1,3,0,0,3,0,3,1,2
1,10000058,-1,-1,-1,-1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,100006,30.8,163,104,160,63,44,0,0,1,2,1,3,1,1,-1,3,2,0,0
2,10000025,1,-1,-1,-1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,100015,24.5,156,88,146,78,73,0,-1,1,1,1,0,1,-1,1,0,0,3,2
3,10000040,-1,-1,-1,1,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,100008,27.7,167,79,117,66,48,0,0,2,2,1,1,0,0,2,3,-1,1,2
4,10000076,-1,-1,-1,-1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,100001,31.2,167,98,128,100,62,0,0,0,1,2,2,1,0,0,-1,-1,2,1


✅ Disease Risk Warehouse: 300 แถว


,member_id,ภาวะเสี่ยง,ระดับความเสี่ยง,คะแนน
0,10000036.0,ความดันโลหิตสูง,สูง,3.000000
1,10000036.0,เบาหวาน (FINDRISC),ปานกลาง,10.000000
2,10000036.0,โรคหัวใจและหลอดเลือด (Simplified),สูง,19.936826
3,10000058.0,ความดันโลหิตสูง,สูง,3.000000
4,10000058.0,เบาหวาน (FINDRISC),สูง,13.000000


### Exercise 3.5: Disease Risk Warehouse

**โจทย์**: สร้างตารางความเสี่ยงโรค 3 ประเภท

**ความเสี่ยงที่ต้องคำนวณ**:
1. **โรคความดันโลหิตสูง**: จากค่า sys/dias
2. **เสี่ยงโรคเบาหวาน**: ใช้ FINDRISC algorithm (คำนวณจาก age, gender, BMI, waistline, sys/dias, relatives_diabetes)
3. **เสี่ยงโรคหัวใจและหลอดเลือด**: คำนวณจาก age, gender, smoking, diabetes, sys, waist/height ratio

**โครงสร้างข้อมูล**:
- แต่ละสมาชิกจะมี 3 แถว (1 แถวต่อ 1 ประเภทความเสี่ยง)
- คอลัมน์: member_id, ภาวะเสี่ยง, ระดับความเสี่ยง, คะแนน

**Hint**: ดูตัวอย่างจาก detailed_health_warehouse_tutorial.ipynb

In [41]:
# ดึงข้อมูลล่าสุด
latest_profiles = ncds_profile.sort_values('dateadd').groupby('memberid').tail(1)
latest_records = ncds_record.sort_values('dateadd').groupby('memberid').tail(1)
latest_habits = ncds_habit.sort_values('dateadd').groupby('memberid').tail(1)
latest_memberdata = memberdata.sort_values(['memberid', 'name', 'dateadd']).groupby(['memberid', 'name']).tail(1)

# ฟังก์ชันดึงค่าจาก memberdata
def get_value(member_id, name):
    try:
        result = latest_memberdata[(latest_memberdata['memberid'] == member_id) &
                                  (latest_memberdata['name'] == name)]['value']
        if len(result) > 0:
            value = result.iloc[0]
            if pd.notna(value) and str(value) not in ['nan', '- - -', '', 'null']:
                return str(value) if name == 'sys/dias' else float(value)
        return None
    except:
        return None

# แยกความดัน sys/dias
def parse_bp(bp):
    if pd.isna(bp) or str(bp) in ['nan', 'None', '- - -', '']: return None, None
    try:
        parts = str(bp).strip().split('/')
        if len(parts) == 2:
            sys = float(parts[0].strip()) if parts[0].strip().replace('.', '').isdigit() else None
            dias = float(parts[1].strip()) if parts[1].strip().replace('.', '').isdigit() else None
            return sys, dias
    except:
        pass
    return None, None

# ระดับความดัน
def bp_status(sys, dias):
    if pd.isna(sys) or pd.isna(dias): return None
    sys_level = "ปกติ" if 50 < sys < 130 else "สูง" if 130 <= sys < 140 else "ค่อนข้างสูง" if 140 <= sys < 180 else "สูงมาก" if sys >= 180 else None
    dias_level = "ปกติ" if 50 < dias < 85 else "สูง" if 85 <= dias < 90 else "ค่อนข้างสูง" if 90 <= dias < 110 else "สูงมาก" if dias >= 110 else None
    if sys_level == "สูงมาก" or dias_level == "สูงมาก": return "สูงมาก"
    if sys_level == "ค่อนข้างสูง" or dias_level == "ค่อนข้างสูง": return "ค่อนข้างสูง"
    if sys_level == "สูง" or dias_level == "สูง": return "สูง"
    if sys_level == "ปกติ" or dias_level == "ปกติ": return "ปกติ"
    return None

# คะแนนเสี่ยงเบาหวาน (FINDRISC)
def diabetes_risk(age, gender, bmi, waist, sys, dias, rel_dm):
    if pd.isna(age) or pd.isna(gender): return None, None
    score, prob = 0, 0

    # อายุ
    if age < 45: score += 1; prob += 1 * -0.0702134
    elif age < 60: score += 1; prob += 1 * 0.2718858
    else: score += 1; prob += 1 * 0.6043599

    # เพศ
    if gender == 1: score += 2; prob += 2 * 0.4422573

    # BMI
    if not pd.isna(bmi):
        if 23 <= bmi < 27.5: score += 1; prob += 1 * 0.6958621
        elif bmi >= 27.5: score += 1; prob += 1 * 1.235097

    # รอบเอว
    if not pd.isna(waist):
        if (gender == 1 and waist >= 90) or (gender == 0 and waist >= 80):
            score += 1; prob += 1 * 0.5567118

    # ความดัน
    if not pd.isna(sys) and not pd.isna(dias):
        if sys >= 140 or dias >= 90: score += 1; prob += 1 * 0.6409517

    # ญาติเบาหวาน
    if not pd.isna(rel_dm) and rel_dm == 1: score += 1; prob += 1 * 1.081356

    # คำนวณเปอร์เซ็นต์
    exp_val = np.exp(prob + -3.580397)
    percentage = (exp_val / (1 + exp_val)) * 100
    return score, percentage

# ระดับเสี่ยงเบาหวาน
def dm_level(score):
    if pd.isna(score): return None
    if score <= 2: return "ปกติ"
    if score <= 5: return "สูง"
    if score <= 8: return "ค่อนข้างสูง"
    return "สูงมาก"

# คะแนนเสี่ยงหัวใจ-หลอดเลือด
def cardio_risk(age, gender, smoking, dm, sbp, waist, height):
    if any(pd.isna(v) for v in [age, gender, sbp, waist, height]): return None, None
    sex = 1 if gender == 1 else 0
    smoke = 1 if smoking == 1 else 0
    dm_val = 1 if dm == 1 else 0
    score = (0.079 * age) + (0.128 * sex) + (0.019350987 * sbp) + (0.58454 * dm_val) + (3.512566 * (waist / height)) + (0.459 * smoke)
    percentage = (1 - (0.978296 ** np.exp(score - 7.720484))) * 100
    return score, percentage

# ระดับเสี่ยงหัวใจ
def cardio_level(pct):
    if pd.isna(pct): return None
    if pct < 10: return "ปกติ"

---

## 📤 Part 4: อัปโหลดไปยัง Google Sheets (15 นาที)

### Exercise 4.1: Authentication

**โจทย์**: Authenticate กับ Google เพื่อใช้งาน Google Sheets API

In [43]:
# TODO: Authenticate กับ Google
# Hint: ใช้ from google.colab import auth
#       auth.authenticate_user()
#       แล้ว authorize กับ gspread

# เขียนโค้ดที่นี่
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

print("✅ Authentication สำเร็จ")

✅ Authentication สำเร็จ


In [49]:
# Code to create a new Google Sheet
spreadsheet_title = 'Health Data Warehouse (ฉบับนักศึกษา)'
try:
    spreadsheet = gc.open(spreadsheet_title)
    print(f"✅ Google Sheet '{spreadsheet_title}' already exists.")
except gspread.SpreadsheetNotFound:
    spreadsheet = gc.create(spreadsheet_title)
    print(f"✅ Google Sheet '{spreadsheet_title}' created successfully.")

✅ Google Sheet 'Health Data Warehouse (ฉบับนักศึกษา)' already exists.


### Exercise 4.3: อัปโหลดข้อมูล

**โจทย์**: อัปโหลดข้อมูลทั้ง 5 warehouses ไปยัง Google Sheets โดยแต่ละ warehouse เป็น 1 sheet

**ชื่อ Sheet**:
- 01-ข้อมูลผู้ตรวจสุขภาพ
- 02-ประวัติโรคNCDs
- 03-พฤติกรรมการกิน
- 04-พฤติกรรมรวม
- 05-ความเสี่ยงโรค

In [51]:
def upload_to_sheet(spreadsheet, df, sheet_name):
    """อัปโหลด DataFrame ไปยัง Google Sheet"""
    # TODO: เขียนโค้ดอัปโหลด
    # Hint: 1. หา/สร้าง worksheet
    #       2. ล้างข้อมูลเดิม (worksheet.clear())
    #       3. แปลง DataFrame เป็น list (ระวัง date objects!)
    #       4. อัปโหลด (worksheet.update())

    # เขียนโค้ดที่นี่
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows="1", cols="1")

    worksheet.clear()
    data_list = [df.columns.values.tolist()] + df.values.tolist()
    worksheet.update(data_list)
    print(f"✅ อัปโหลดข้อมูล '{sheet_name}' สำเร็จ")


# TODO: อัปโหลดทั้ง 5 warehouses
# เขียนโค้ดที่นี่
# Replace with your Google Sheet title
spreadsheet_title = 'Health Data Warehouse (ฉบับนักศึกษา)'
spreadsheet = gc.open(spreadsheet_title)

upload_to_sheet(spreadsheet, health_profile_wh, '01-ข้อมูลผู้ตรวจสุขภาพ')
upload_to_sheet(spreadsheet, ncds_disease_wh, '02-ประวัติโรคNCDs')
upload_to_sheet(spreadsheet, eating_behavior_wh, '03-พฤติกรรมการกิน')
upload_to_sheet(spreadsheet, comprehensive_behavior_wh, '04-พฤติกรรมรวม')
upload_to_sheet(spreadsheet, disease_risk_wh, '05-ความเสี่ยงโรค')

✅ อัปโหลดข้อมูล '01-ข้อมูลผู้ตรวจสุขภาพ' สำเร็จ
✅ อัปโหลดข้อมูล '02-ประวัติโรคNCDs' สำเร็จ
✅ อัปโหลดข้อมูล '03-พฤติกรรมการกิน' สำเร็จ
✅ อัปโหลดข้อมูล '04-พฤติกรรมรวม' สำเร็จ
✅ อัปโหลดข้อมูล '05-ความเสี่ยงโรค' สำเร็จ


In [53]:
print(spreadsheet.url)

https://docs.google.com/spreadsheets/d/1HPSFdcywjMqlSbSLezsdRcvsGwyPYxoI5GyfPB9q9m0


---

## 📝 Part 5: สรุปและคำถาม (10 นาที)

### คำถามท้ายบท:

**Q1**: ทำไมต้องดึงข้อมูล "ล่าสุด" ของแต่ละสมาชิก ไม่ใช่ข้อมูลทั้งหมด?

**คำตอบ**: _______________________________________________________

---

**Q2**: ในตาราง NCDs Disease ทำไมถึงมีหลายแถวต่อ 1 สมาชิก?

**คำตอบ**: _______________________________________________________

---

**Q3**: อะไรคือความแตกต่างระหว่าง "_score" และ "label" ในตาราง Eating Behavior?

**คำตอบ**: _______________________________________________________

---

**Q4**: ลองคิดว่าถ้าจะเพิ่ม Warehouse ตารางที่ 6 ควรจะเป็นตารางอะไร? และมีข้อมูลอะไรบ้าง?

**คำตอบ**: _______________________________________________________

---

## ✅ Checklist สำหรับการส่งงาน

- [ ] สร้าง Health Profile Warehouse ได้อย่างน้อย 100 แถว
- [ ] สร้าง NCDs Disease Warehouse ได้
- [ ] สร้าง Eating Behavior Warehouse ได้
- [ ] สร้าง Comprehensive Behavior Warehouse ได้
- [ ] สร้าง Disease Risk Warehouse ได้
- [ ] อัปโหลดข้อมูลไปยัง Google Sheets สำเร็จ
- [ ] ตอบคำถามท้ายบทครบทั้ง 4 ข้อ
- [ ] แชร์ Google Sheets ให้อาจารย์ดูได้

## 📤 วิธีการส่งงาน

1. Save notebook นี้ (Download .ipynb)
2. Copy URL ของ Google Sheets ที่สร้าง
3. ส่งทั้ง 2 อย่างใน Google Classroom

---

**Google Sheets URL**: _______________________________________________________

**หมายเหตุ**: กรุณาแชร์ Google Sheets ให้อาจารย์สามารถดูได้ (Viewer)

---

## 🎓 เกณฑ์การให้คะแนน (100 คะแนน)

| หัวข้อ | คะแนน |
|--------|-------|
| Part 1-2: เตรียมสภาพแวดล้อมและดึงข้อมูล | 15 |
| Part 3.1: Health Profile Warehouse | 20 |
| Part 3.2: NCDs Disease Warehouse | 12 |
| Part 3.3: Eating Behavior Warehouse | 12 |
| Part 3.4: Comprehensive Behavior Warehouse | 12 |
| Part 3.5: Disease Risk Warehouse | 19 |
| Part 4: อัปโหลดไปยัง Google Sheets | 5 |
| Part 5: คำถามท้ายบท | 5 |
| **รวม** | **100** |

---

**สิ่งที่ได้เรียนรู้**:
✅ การทำ Data Transformation  
✅ การใช้ Pandas สำหรับ Data Wrangling  
✅ การสร้าง Data Warehouse  
✅ การคำนวณความเสี่ยงโรค (FINDRISC, Cardiovascular Risk)  
✅ การใช้ Google Sheets API  

**Good Luck! 🎓**

In [ ]:
# ปิดการเชื่อมต่อ Database
conn.close()
print("✅ ปิดการเชื่อมต่อ Database เรียบร้อยแล้ว")